In [373]:
import pandas as pd

df = pd.read_csv("train.csv")
dftest = pd.read_csv("test.csv")

In [374]:
df.head(3)

,Unnamed: 0,Weight,Age,Days,Vegan,Cats,Cash,Size,Toppings
0,1,106.238809,36.596211,38,0,1,5.699125,No order,No order
1,2,184.378192,28.739952,28,0,0,1.171537,No order,No order
2,3,232.475732,106.605562,38,1,1,259.440103,Large,Hawaiian


In [375]:
from sklearn import preprocessing

Size = df.iloc[:,7:8]

le = preprocessing.LabelEncoder()
le.fit(Size)
encoded_Size = le.transform(Size)

Toppings = df.iloc[:,8:9]

le1 = preprocessing.LabelEncoder()
le1.fit(Toppings)
encoded_Toppings = le1.transform(Toppings)

df['Size'] = encoded_Size
df['Toppings'] = encoded_Toppings

In [450]:
train_X = df.iloc[:,:7]
train_Y = df.iloc[:,7:9]

In [398]:
train_X.head(3)

,Unnamed: 0,Weight,Age,Days,Vegan,Cats,Cash
0,1,106.238809,36.596211,38,0,1,5.699125
1,2,184.378192,28.739952,28,0,0,1.171537
2,3,232.475732,106.605562,38,1,1,259.440103


In [378]:
train_Y.head(3)

,Size,Toppings
0,3,4
1,3,4
2,1,2


In [65]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cross_validation import train_test_split
from sklearn import metrics

mlb = MultiLabelBinarizer()
y_enc = mlb.fit_transform(train_Y.values.tolist())
# print(len(train_Y), len(y_enc))
print(y_enc)

clf = OneVsRestClassifier(SVC(probability=True))
clf.fit(train_X, y_enc)
hyp = clf.predict(dftest)

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 1 0 ... 0 1 0]]


In [105]:
# print(hyp)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_X, train_Y)
# clf.predict(dftest)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [448]:
X = df.iloc[:,:7]
Y = df.iloc[:,7:9]

split = int(len(X) * 0.8)
train_X = X[:split]
train_Y = Y[:split]
test_X = X[split:]
test_Y = Y[split:]


clf = RandomForestClassifier()
clf.fit(train_X, train_Y)
hyp = clf.predict(test_X)
accuracy = np.sum(hyp == test_Y)

print("Accuracy Score :", accuracy/100)

file = open('pred2.txt','w')

for i in range(len(hyp)):
    file.write('\n'+str(i))
    file.write(','+str(le.inverse_transform(int(hyp[i][0]))))
    file.write(','+str(le1.inverse_transform(int(hyp[i][1]))))

file.close()

Accuracy Score : Size        0.36
Toppings    0.42
dtype: float64


In [451]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import warnings; warnings.simplefilter('ignore')
import numpy as np
from sklearn import tree

print(len(train_X), len(train_Y))

listacc = []

x_for_kfold = train_X
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(x_for_kfold):
#     print('Train: {} | Test: {}'.format(train_index,test_index))
    X_train, X_test = train_X.values[train_index], train_X.values[test_index]
    y_train, y_test = train_Y.values[train_index], train_Y.values[test_index]
#     RandomForestClassifier
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    hyp = clf.predict(X_test)
    accuracy = np.sum(hyp == y_test)
#     print(hyp == y_test)
#     print(len(hyp))
    print('Accuracy : ', accuracy/100)
    e = np.equal(hyp ,y_test)
#     Decision tree Classifier
#     decision_tree = tree.DecisionTreeClassifier()
#     decision_tree.fit(X_train, y_train)
#     hyp = decision_tree.predict(X_test)
#     accuracy = np.sum(hyp == y_test)
#     print('Accuracy : ', accuracy/100)
#     print(accuracy/len(y_test))
    listacc.append(accuracy)
print(np.sum(listacc)/10)

500 500
Accuracy :  0.3
Accuracy :  0.3
Accuracy :  0.48
Accuracy :  0.39
Accuracy :  0.42
Accuracy :  0.36
Accuracy :  0.32
Accuracy :  0.45
Accuracy :  0.37
Accuracy :  0.39
37.8


In [351]:
import pandas as pd

df = pd.read_csv("aggregated.csv")

In [352]:
df.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE,Unnamed: 13
0,2.0,6.0,2017-02-25,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1000.0,0.0,156.0,937.0,NaN
1,2.0,7.0,2017-02-26,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0,NaN
2,2.0,1.0,2017-02-27,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1028.0,0.0,158.0,937.0,NaN


In [353]:
# print(len(df))
df = df.iloc[:,:-1] #Ignoring last column (Unname: 13)
print(len(df))
# df = df.dropna()
# print(len(df))

a = df['ARR_DEL15'] != 0
b = df[df['ARR_DEL15'] != 1]
c = df[df['ARR_DEL15'] != 0]

5129354


In [355]:
import numpy as np

totaldelays = np.sum(a)
# totalnodelays = np.sum(b)
# print(totaldelays, totalnodelays)

b = b.iloc[:totaldelays,:]
c.head(3)


,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
9,2.0,1.0,2017-02-06,B6,33.0,BTV,"Burlington, VT",JFK,"New York, NY",1907.0,1.0,90.0,266.0
11,2.0,3.0,2017-02-08,B6,33.0,BTV,"Burlington, VT",JFK,"New York, NY",1907.0,1.0,90.0,266.0
12,2.0,4.0,2017-02-09,B6,33.0,BTV,"Burlington, VT",JFK,"New York, NY",1907.0,NaN,90.0,266.0


In [357]:
frames = [b, c]
result = pd.concat(frames)
len(result)
result.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
0,2.0,6.0,2017-02-25,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1000.0,0.0,156.0,937.0
1,2.0,7.0,2017-02-26,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0
2,2.0,1.0,2017-02-27,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1028.0,0.0,158.0,937.0


In [358]:
from sklearn import preprocessing

FL_DATE = result.iloc[:,2]
UNIQUE_CARRIER = result.iloc[:,3]
ORIGIN = result.iloc[:,5]
ORIGIN_CITY_NAME = result.iloc[:,6]
DEST = result.iloc[:,7]
DEST_CITY_NAME = result.iloc[:,8]
# UNIQUE_CARRIER.head(5), ORIGIN.head(5), ORIGIN_CITY_NAME.head(5), DEST.head(5), DEST_CITY_NAME.head(5)

In [359]:
le = preprocessing.LabelEncoder()

le.fit(FL_DATE)
encoded_FL_DATE = le.transform(FL_DATE)

le.fit(UNIQUE_CARRIER)
encoded_UNIQUE_CARRIER = le.transform(UNIQUE_CARRIER)

le.fit(ORIGIN)
encoded_ORIGIN = le.transform(ORIGIN)

le.fit(ORIGIN_CITY_NAME)
encoded_ORIGIN_CITY_NAME = le.transform(ORIGIN_CITY_NAME)

le.fit(DEST)
encoded_DEST = le.transform(DEST)

le.fit(DEST_CITY_NAME)
encoded_DEST_CITY_NAME = le.transform(DEST_CITY_NAME)

result['FL_DATE'] = encoded_FL_DATE
result['UNIQUE_CARRIER'] = encoded_UNIQUE_CARRIER
result['ORIGIN'] = encoded_ORIGIN
result['ORIGIN_CITY_NAME'] = encoded_ORIGIN_CITY_NAME
result['DEST'] = encoded_DEST
result['DEST_CITY_NAME'] = encoded_DEST_CITY_NAME

In [360]:
result.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
0,2.0,6.0,330,2,28.0,188,218,98,207,1000.0,0.0,156.0,937.0
1,2.0,7.0,331,2,28.0,188,218,98,207,739.0,0.0,153.0,937.0
2,2.0,1.0,332,2,28.0,188,218,98,207,1028.0,0.0,158.0,937.0


In [366]:
# Splitting into train data and train labels
X = result.iloc[:,result.columns != 'ARR_DEL15']
# X = X.iloc[:,:-1] #Ignoring last column (Unname: 13)
Y = result.iloc[:,result.columns == 'ARR_DEL15']

split = int(len(Y) * 0.8)
train_X = X[:split]
train_Y = Y[:split]
test_X = X[split:]
test_Y = Y[split:]

from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
train_x_imputed = imputer.fit_transform(train_X)
test_x_imputed = imputer.fit_transform(test_X)

train_y_imputed = imputer.fit_transform(train_Y)
test_y_imputed = imputer.fit_transform(test_Y)

In [367]:
clf = RandomForestClassifier()
clf.fit(train_x_imputed, train_y_imputed)
hyp = clf.predict(test_x_imputed)

In [368]:
from sklearn.metrics import accuracy_score

accuracy_score(hyp, test_y_imputed)

0.5282063553076721

In [369]:
from sklearn.metrics import accuracy_score, confusion_matrix

print('Confusion Matrix:\n', confusion_matrix(test_y_imputed, hyp))

Confusion Matrix:
 [[     0      0]
 [185338 207499]]


In [440]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf,train_x_imputed,train_y_imputed,cv=10)
scores

array([0.65120438, 0.17622427, 0.13549496, 0.18501289, 0.17951443,
       0.2507605 , 0.31913526, 0.1765945 , 0.18719691, 0.46065142])